In [1]:
import numpy as np

with open('1268-0.txt', 'r') as fp:
  text = fp.read()

start_idx = text.find('THE MYSTERIOUS ISLAND')
end_idx = text.find('End of the Project Gutenberg')
text = text[start_idx:end_idx]
char_set = set(text)
print('Total Length: ', len(text))

Total Length:  1130711


In [3]:
!ls

1268-0.txt                      examples
1st_flask_app_1                 figpath.svg
ML10.ipynb                      figpath2.png
ML11.ipynb                      index.html
ML13.ipynb                      kaggle
ML16-2.ipynb                    kagglebook-master
ML3.ipynb                       machineLearning.ipynb
ML6.ipynb                       machineLearning2.ipynb
ML8.ipynb                       ml
ML9.ipynb                       mnist_scaled.npz
Pmaster                         movie_data.csv
README.md                       movieclassifier
RStudio                         my-test-env
aclImdb                         mydata.csv
array_archive.npz               mydata.h5
array_compresed.npz             mydata.sqlite
cat_dog_images                  numpy01.py
chapter 10.ipynb                plot_decision_regions_script.py
chapter1.ipynb                  present A.ipynb
chapter11.ipynb                 push.command
chapter12.ipynb                 push_from_main.command
chapter13.1.ipynb   

In [2]:
print('Unique Charactor: ',len(char_set))

Unique Charactor:  85


In [3]:
chars_sorted = sorted(char_set)
char2int = {ch:i for i, ch in enumerate(chars_sorted)}
char_array = np.array(chars_sorted)
text_encoded = np.array([char2int[ch] for ch in text], dtype=np.int32)
print('Text encoded shape: ', text_encoded.shape)

Text encoded shape:  (1130711,)


In [4]:
print(text[:15], '== Encodind ==>', text_encoded[:15])

THE MYSTERIOUS  == Encodind ==> [48 36 33  1 41 53 47 48 33 46 37 43 49 47  1]


In [5]:
print(text_encoded[15:21], '== Reverse ==>', ''.join(char_array[text_encoded[15:21]]))

[37 47 40 29 42 32] == Reverse ==> ISLAND


In [6]:
import tensorflow as tf
ds_text_encoded = tf.data.Dataset.from_tensor_slices(text_encoded)
for ex in ds_text_encoded.take(5):
    print('{} -> {}'.format(ex.numpy(), char_array[ex.numpy()]))

48 -> T
36 -> H
33 -> E
1 ->  
41 -> M


2022-09-15 19:01:05.095477: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [7]:
seq_length = 40
chunk_size = seq_length+1
ds_chunks = ds_text_encoded.batch(chunk_size, drop_remainder=True)
def split_input_target(chunk):
    input_seq = chunk[:-1]
    target_seq = chunk[1:]
    return input_seq, target_seq

ds_sequences = ds_chunks.map(split_input_target)

In [9]:
for example in ds_sequences.take(2):
    print(' Input(x): ', repr(''.join(char_array[example[0].numpy()])))
    print('Target (y): ', repr(''.join(char_array[example[1].numpy()])))

 Input(x):  'THE MYSTERIOUS ISLAND ***\n\n\n\n\nTHE MYSTER'
Target (y):  'HE MYSTERIOUS ISLAND ***\n\n\n\n\nTHE MYSTERI'
 Input(x):  'OUS ISLAND\n\nby Jules Verne\n\n1874\n\n\n\n\nPAR'
Target (y):  'US ISLAND\n\nby Jules Verne\n\n1874\n\n\n\n\nPART'


In [10]:
BATCH_SIZE  = 64
BUFFER_SIZE = 10000
ds = ds_sequences.shuffle(BUFFER_SIZE).batch(BATCH_SIZE)

In [11]:
def build_model(vocab_size, embedding_dim, rnn_units):
    model = tf.keras.Sequential([
        tf.keras.layers.Embedding(vocab_size, embedding_dim), 
        tf.keras.layers.LSTM(rnn_units, return_sequences=True),
        tf.keras.layers.Dense(vocab_size)
    ])
    return model

charset_size = len(char_array)
embedding_dim = 256
rnn_units = 512
tf.random.set_seed(1)
model = build_model(vocab_size=charset_size, embedding_dim = embedding_dim, rnn_units=rnn_units)
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, None, 256)         21760     
                                                                 
 lstm (LSTM)                 (None, None, 512)         1574912   
                                                                 
 dense (Dense)               (None, None, 85)          43605     
                                                                 
Total params: 1,640,277
Trainable params: 1,640,277
Non-trainable params: 0
_________________________________________________________________


In [12]:
model.compile(optimizer='adam', loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True))

In [13]:
model.fit(ds, epochs=10)

Epoch 1/10
431/431 [==============================] - 102s 232ms/step - loss: 2.3364
Epoch 2/10
431/431 [==============================] - 104s 239ms/step - loss: 1.7606
Epoch 3/10
431/431 [==============================] - 105s 241ms/step - loss: 1.5569
Epoch 4/10
431/431 [==============================] - 103s 238ms/step - loss: 1.4396
Epoch 5/10
431/431 [==============================] - 124s 286ms/step - loss: 1.3651
Epoch 6/10
431/431 [==============================] - 122s 279ms/step - loss: 1.3128
Epoch 7/10
431/431 [==============================] - 106s 245ms/step - loss: 1.2730
Epoch 8/10
431/431 [==============================] - 112s 258ms/step - loss: 1.2423
Epoch 9/10
431/431 [==============================] - 102s 235ms/step - loss: 1.2155
Epoch 10/10
431/431 [==============================] - 101s 234ms/step - loss: 1.1926


In [14]:
tf.random.set_seed(1)
logits = [[1.0, 1.0, 1.0]]
print('Probabilities:', tf.math.softmax(logits).numpy()[0])

Probabilities: [0.33333334 0.33333334 0.33333334]


In [16]:
samples = tf.random.categorical(logits=logits, num_samples=10)
tf.print(samples.numpy())

array([[0, 0, 1, 2, 0, 0, 0, 0, 1, 0]])


In [17]:
tf.random.set_seed(1)
logits = [[1.0, 1.0, 3.0]]
print('Probabilities: ', tf.math.softmax(logits).numpy()[0])

Probabilities:  [0.10650698 0.10650698 0.78698605]


In [19]:
samples = tf.random.categorical(logits=logits, num_samples=10)
tf.print(samples.numpy())

array([[2, 0, 2, 2, 2, 0, 1, 2, 2, 0]])


In [28]:
def sample(model, starting_str, len_generated_text=500, max_input_length = 40, scale_factor=1.0):
    encoded_input = [char2int[s] for s in starting_str]
    encoded_input = tf.reshape(encoded_input, (1,-1))
    
    generated_str = starting_str
    
    model.reset_states()
    for i in  range(len_generated_text):
        logits = model(encoded_input)
        logits = tf.squeeze(logits, 0)
        
        scaled_logits = logits * scale_factor
        new_char_idx = tf.random.categorical(scaled_logits, num_samples=1)
        new_char_idx = tf.squeeze(new_char_idx)[-1].numpy()
        generated_str += str(char_array[new_char_idx])
        new_char_idx = tf.expand_dims([new_char_idx], 0)
        encoded_input = tf.concat([encoded_input, new_char_idx], axis=1)
        encoded_input = encoded_input[:, -max_input_length:]
        
    return generated_str

In [23]:
tf.random.set_seed(1)
print(sample(model, starting_str='The island'))

tf.Tensor([[48 62 59  1 63 73 66 55 68 58]], shape=(1, 10), dtype=int32)
The island is our bead up, and the work his shailed
sight
began to say, the number of
June his cheeks to piece sharm nothing to give an hour grunt and
all the wished on the electrory relished it in which they saw that the cart without death to the operation of one of the house, belonged, could leeping up from her, and
he bed of
Uneorably that Spilett’s rays had been lovely to me.”

“And when the
arch till, Pencroft, were it not from this point mawnessed. The dails were to be knowledged a word: Lincoln Isl


In [24]:
logits = np.array([[1.0, 1.0, 3.0]])
print('Probabilities before scaling: ', tf.math.softmax(logits).numpy()[0])

Probabilities before scaling:  [0.10650698 0.10650698 0.78698604]


In [25]:
print('Probabilities after scaling with 0.5: ', tf.math.softmax(0.5*logits).numpy()[0])

Probabilities after scaling with 0.5:  [0.21194156 0.21194156 0.57611688]


In [26]:
print('Probabilities after scaling with 0.1: ', tf.math.softmax(0.1 *logits).numpy()[0])

Probabilities after scaling with 0.1:  [0.31042377 0.31042377 0.37915245]


In [27]:
tf.random.set_seed(1)
print(sample(model, starting_str='The island', scale_factor=2.0))

tf.Tensor([[48 62 59  1 63 73 66 55 68 58]], shape=(1, 10), dtype=int32)
The island was simple at least to be some motionless sent and seen that the rock was a sort of sea, which in the convicts were on the shore of the colonists were to be found in the signal to do an island with the sea, which
the trateat of the forest with start in the engineer, the settlers were not even destroyed the animal point of the subject could, that is to say, the situation of the lake, said the profession of the corral was profter in the neight of the cart was to be seen to the corral. From time t


In [29]:
tf.random.set_seed(1)
print(sample(model, starting_str = 'The island', scale_factor=0.5))

The island harng capa.
Weau; 7Natiasquin;, word opbark, safe!-- housaipseliquun.
Gide.
The asted kilo0m, ildeplue no proristence. Gifes far, grying
jacamaritlesstsla, mile! Har?r Would drook Uly has’ kill. Po.2
found sprtaf
clecticulce Harding tolse. Uso, peneplation, anr; there-botainst his admirenhistence.”

1er.
FH Neial,
jo af its twon Masboarmy;” re will
aquered!” Twolveople’c natious, sowe
/salend, we end, for this
frigtfads. Yousavel? Had
he whuensif-Scabforuely?” fifthered howes- yeighb, Harding, 
